In [2]:
import csv

def csv_to_dict(csv_file):
    data_dict = {}
    id_list = []
    
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            #print(row)
            data_dict[int(row[0])] = int(float(row[2]))  # Convert QTY to integer if needed
            id_list.append(int(row[0]))
    
    return data_dict, id_list

def csv_to_distance_dict(csv_file):
    distance_dict = {}
    
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            key = (int(row[0]), int(row[1]))
            distance_dict[key] = int(float(row[2]))  # Convert Distance to float if needed
    
    return distance_dict




import pyomo.environ as pyo



supply, suppliers = csv_to_dict('SupplierData.csv')
demand, counties = csv_to_dict('CountyData.csv')

#print(counties)
# Distance matrix (distance from supplier to county)
distances = csv_to_distance_dict("DistanceListShort.csv")
#print(distance)
# Truck capacity
truck_capacity = 16000

# Cost per unit-mile
cost_per_unit_mile = .156

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.suppliers = pyo.Set(initialize=suppliers)
model.counties = pyo.Set(initialize=counties)

# Define parameters
pyo.Param()
model.supply = pyo.Param(model.suppliers, initialize=supply)
model.demand = pyo.Param(model.counties, initialize=demand)
model.distance = pyo.Param(model.suppliers, model.counties, initialize=distances)
model.truck_capacity = pyo.Param(initialize=truck_capacity)
model.cost_per_unit_mile = pyo.Param(initialize=cost_per_unit_mile)

# Define decision variables
model.x = pyo.Var(model.suppliers, model.counties, domain=pyo.NonNegativeReals)  # Amount shipped from supplier to county
model.trucks = pyo.Var(model.suppliers, model.counties, domain=pyo.NonNegativeIntegers)  # Number of trucks used

# Objective: Minimize total cost
def total_cost_rule(model):
    return sum(
        model.cost_per_unit_mile * model.distance[s, c] * model.x[s, c]
        for s in model.suppliers
        for c in model.counties
    )

model.total_cost = pyo.Objective(rule=total_cost_rule, sense=pyo.minimize)

# Constraints
# 1. Supply constraint: Total shipped from each supplier <= supply
def supply_constraint_rule(model, s):
    return sum(model.x[s, c] for c in model.counties) <= model.supply[s]

model.supply_constraint = pyo.Constraint(model.suppliers, rule=supply_constraint_rule)

# 2. Demand constraint: Total shipped to each county >= demand
def demand_constraint_rule(model, c):
    return sum(model.x[s, c] for s in model.suppliers) >= model.demand[c]

model.demand_constraint = pyo.Constraint(model.counties, rule=demand_constraint_rule)

# 3. Truck capacity constraint: Amount shipped <= number of trucks * truck capacity
def truck_capacity_constraint_rule(model, s, c):
    return model.x[s, c] <= model.trucks[s, c] * model.truck_capacity

model.truck_capacity_constraint = pyo.Constraint(model.suppliers, model.counties, rule=truck_capacity_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('gurobi')  # Use GLPK solver (open-source)
results = solver.solve(model)

# Print results
print("Objective Value (Total Cost):", pyo.value(model.total_cost))
print("\nShipments:")
for s in model.suppliers:
    for c in model.counties:
        if pyo.value(model.x[s, c]) > 0:
            print(f"Ship {pyo.value(model.x[s, c]):.2f} units from {s} to {c} using {pyo.value(model.trucks[s, c])} trucks")

Objective Value (Total Cost): 700246553.604

Shipments:
Ship 1387789.00 units from 0 to 51 using 87.0 trucks
Ship 4008876.00 units from 0 to 76 using 251.0 trucks
Ship 2328785.00 units from 2 to 60 using 146.0 trucks
Ship 2851777.00 units from 2 to 65 using 179.0 trucks
Ship 3096465.00 units from 2 to 81 using 194.0 trucks
Ship 7102987.00 units from 3 to 40 using 444.0 trucks
Ship 644626.00 units from 3 to 70 using 41.0 trucks
Ship 1950782.00 units from 3 to 83 using 122.0 trucks
Ship 2739378.00 units from 4 to 59 using 172.0 trucks
Ship 925730.00 units from 4 to 85 using 58.0 trucks
Ship 2681959.00 units from 5 to 66 using 168.0 trucks
Ship 99913.00 units from 6 to 48 using 7.0 trucks
Ship 58689.00 units from 6 to 58 using 4.0 trucks
Ship 128194.00 units from 6 to 64 using 9.0 trucks
Ship 307894.00 units from 6 to 86 using 20.0 trucks
Ship 156740.00 units from 6 to 87 using 10.0 trucks
Ship 1986762.00 units from 7 to 26 using 125.0 trucks
Ship 497268.00 units from 7 to 37 using 32.0 t

KeyError: "Index '(26, 27)' is not valid for indexed component 'distance'"